In [ ]:
!pip install tensorflow transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import create_optimizer
from datasets import load_dataset
import numpy as np

In [ ]:
model_checkpoint = "distilbert-base-uncased"
num_labels = 2

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"],  truncation  = True,  max_length = 128, padding = "max_length")

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})


In [ ]:
tf_train_dataset = tokenized_dataset['train'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask'],
    label_cols = ['label'],
    shuffle = True,
    batch_size = 16

)

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
tf_validation_dataset = tokenized_dataset['test'].to_tf_dataset(
    columns = ['input_ids', 'attention_mask'],
    label_cols = ['label'],
    shuffle = False,
    batch_size = 16

)

In [ ]:
num_epochs = 1
batch_size = 16
learning_rate = 5e-5
weight_decay = 0.01
num_train_steps = len(tokenized_dataset['train']) // batch_size * num_epochs

In [ ]:
optimizer, lr_schedule = create_optimizer(
    init_lr = learning_rate,
    num_train_steps = num_train_steps,
    weight_decay_rate = weight_decay,
    num_warmup_steps = 0

)

In [ ]:
model.compile(optimizer = optimizer, loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics = tf.metrics.SparseTopKCategoricalAccuracy()  )

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data = tf_validation_dataset,
    epochs = num_epochs
)

1563/1563 [==============================] - 512s 312ms/step - loss: 0.6401 - sparse_top_k_categorical_accuracy: 1.0000 - val_loss: 0.6909 - val_sparse_top_k_categorical_accuracy: 1.0000


In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
out_dir = '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning'
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)

('/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning/tokenizer_config.json',
 '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning/special_tokens_map.json',
 '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning/vocab.txt',
 '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning/added_tokens.json',
 '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning/tokenizer.json')

In [3]:
out_dir = '/content/drive/MyDrive/Luminar Feb 25/Fine-Tuning'

In [5]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
loaded_model = TFAutoModelForSequenceClassification.from_pretrained(out_dir)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Luminar Feb 25/Fine-Tuning.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [6]:
loaded_tokenizer = AutoTokenizer.from_pretrained(out_dir)

In [33]:
def test_loaded_model(text):
  inputs = loaded_tokenizer(text, padding = True, truncation = True, return_tensors = 'tf' )
  outputs = loaded_model(inputs)
  predictions = tf.nn.softmax(outputs.logits)
  predicted_class = tf.argmax(predictions, axis = 1).numpy()[0]

  sentiment = "Positive" if predicted_class == 1 else "Negative"
  confidence = predictions[0][predicted_class].numpy()
  return sentiment, confidence

In [34]:
test_text = "I thoroughly enjoyed this movie. The acting was superb"
sentiment, confidence = test_loaded_model(test_text)


In [35]:
print(f"Test Text : {test_text}")
print(f"Predicted Sentiment  : {sentiment} (confidence : {confidence})" )

Test Text : I thoroughly enjoyed this movie. The acting was superb
Predicted Sentiment  : Positive (confidence : 0.53773033618927)


#### Evaluating the performance of Fine-Tuned model against Pre-Trained model

In [36]:
model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [45]:
domain_specific_example = "The movie is worst one I ever saw"

In [40]:
import numpy as np

In [46]:
print("Domain Specific example comparison")
print("-" * 80)


inputs = loaded_tokenizer(domain_specific_example, padding = True, truncation = True, return_tensors = 'tf' )

# Get predictions from Pre-Trained Model
outputs_pretrained = model(inputs)
preds_pretrained = tf.nn.softmax(outputs_pretrained.logits).numpy()[0]
pred_class_pretrained = np.argmax(preds_pretrained)
confidence_pretrained = preds_pretrained[pred_class_pretrained]

# Get predictions from Fine-Tuned Model
outputs_finetuned = loaded_model(inputs)
preds_finetuned = tf.nn.softmax(outputs_finetuned.logits).numpy()[0]
pred_class_finetuned = np.argmax(preds_finetuned)
confidence_finetuned = preds_finetuned[pred_class_finetuned]


print(f"Text : {text}")
print(f"Pre-Trained Model : Class {pred_class_pretrained} with {confidence_pretrained: .2f} confidence")
print(f"Fine-Tuned Model : Class {pred_class_finetuned} with {confidence_finetuned: .2f} confidence")






Domain Specific example comparison
--------------------------------------------------------------------------------
Text : c
Pre-Trained Model : Class 1 with  0.52 confidence
Fine-Tuned Model : Class 1 with  0.53 confidence
